## Analyzing Saccharide BGC Glycosyltransferases using Foldseek

As explained [in the docs](../docs/glycosyltransferase_analysis.md), we've searched our Saccharide BGC-associated glycosyltransferases by searching them against CAZY using Foldseek.

Let's load this DataFrame:

In [8]:
import pandas as pd

foldseek_df = pd.read_csv("../metadata/cazy_foldseek_search.tsv", sep="\t").rename(
columns={"query": "Query", "target": "Target", "fident": "Identity", "alnlen": "Aligned Length", "mismatch": "Number of Mismatches",
         "gapopen": "Gap Openings", "qstart": "Query Start", "qend": "Query End", "tstart": "Target Start",
         "tend": "Target End", "evalue": "E-Value", "bits": "TM-Score", "prob": "Probability"})

foldseek_df["Query"] = foldseek_df["Query"].str[:-4]
foldseek_df["Target"] = foldseek_df["Target"].str[:-4]
foldseek_df["TM-Score"] = foldseek_df["TM-Score"] / 100

foldseek_df

,Query,Target,Identity,Aligned Length,Number of Mismatches,Gap Openings,Query Start,Query End,Target Start,Target End,E-Value,TM-Score,Probability
0,MGYP001572126581,5MLZ,0.320,243,146,14,1,235,4,235,0.7405,0.88,0.995
1,MGYP001572126581,4DEC,0.119,252,177,21,1,233,25,250,0.7020,0.76,0.971
2,MGYP001572126581,7QCP,0.131,251,177,19,1,233,35,262,0.7002,0.76,0.971
3,MGYP001572126581,3F1Y,0.113,255,187,19,1,235,33,268,0.6987,0.79,0.981
4,MGYP001572126581,3BCV,0.153,222,139,13,1,201,3,196,0.6967,0.63,0.837
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19888,MGYP003109256094,5MLZ,0.024,406,120,19,49,299,11,295,0.2221,0.21,0.034
19889,MGYP003109256094,7U4H,0.013,731,160,55,1,393,64,571,0.2213,0.25,0.057
19890,MGYP003109256094,3HZS,0.015,257,97,28,232,386,7,209,0.2191,0.16,0.014
19891,MGYP003109256094,7Y4I,0.026,491,155,36,53,355,206,561,0.2018,0.25,0.057


Note that we've got nearly 20,000 hits, because Foldseek just gives us back all hits for each query. Let's narrow this down to only the best hit:

In [9]:
best_hits = foldseek_df.sort_values(by="Probability", ascending=False).groupby("Query").first().reset_index()
best_hits

,Query,Target,Identity,Aligned Length,Number of Mismatches,Gap Openings,Query Start,Query End,Target Start,Target End,E-Value,TM-Score,Probability
0,MGYP001186727001,5VQM,0.096,52,43,3,7,56,39,88,0.3578,0.62,0.817
1,MGYP001196446441,6APL,0.161,236,145,13,1,203,78,293,0.6417,0.74,0.961
2,MGYP001316708597,7SP8,0.081,370,184,23,1,223,41,401,0.5862,0.79,0.981
3,MGYP001324906288,6E4R,0.108,294,191,17,1,239,67,344,0.5730,0.75,0.967
4,MGYP001364174740,6FSN,0.066,317,203,27,2,267,1,275,0.6753,0.68,0.912
...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,MGYP003624946996,7D5K,0.036,463,232,23,1,265,88,534,0.5218,0.74,0.961
94,MGYP003626144850,3S28,0.051,603,298,37,9,343,157,753,0.5677,0.76,0.971
95,MGYP003626144883,3Q3E,0.034,435,143,26,8,199,179,579,0.3804,0.54,0.601
96,MGYP003631920257,6E4R,0.089,323,190,25,1,240,68,369,0.5751,0.76,0.971


Now, let's load our CAZY DataFrame, and add the CAZY family information to our Foldseek hits DataFrame:

In [10]:
cazy_df = pd.read_csv("../metadata/cazy_structures.csv")
cazy_df["Target"] = cazy_df["PDB Code"].str.split("[").str[0]
best_hits = best_hits.merge(cazy_df, on="Target")
best_hits

,Query,Target,Identity,Aligned Length,Number of Mismatches,Gap Openings,Query Start,Query End,Target Start,Target End,...,Probability,PDB Code,Ligand,Resolution,Protein Name,EC#,Organism,GenBank,Uniprot,Family
0,MGYP001186727001,5VQM,0.096,52,43,3,7,56,39,88,...,0.817,"5VQM[A,B]",NaN,2.79,CDR20291_0582 (TcdB),NaN,Clostridioides difficile R20291,CBE02479.1AVB36394.1,NaN,44
1,MGYP001196446441,6APL,0.161,236,145,13,1,203,78,293,...,0.961,"6APL[A,B,C,D,E,F]",NaN,2.35,"CMP-NeuAc:N-acetylgalactosaminide α-2,6-sialyl...",2.4.3.3,Homo sapiens,AAH40455.1AAA52228.1AAV38775.1CAB61434.1CAS916...,A0A024R8M1Q9UJ37,29
2,MGYP001316708597,7SP8,0.081,370,184,23,1,223,41,401,...,0.981,7SP8[A],NaN,2.70,hyaluronan synthase (PBCVCZ2_118R;Cz-2_118R),2.4.1.212,Paramecium bursaria Chlorella virus CZ-2,AGE52748.1,M1H2Q1,2
3,MGYP001379529187,7SP8,0.074,374,186,24,1,223,41,405,...,0.984,7SP8[A],NaN,2.70,hyaluronan synthase (PBCVCZ2_118R;Cz-2_118R),2.4.1.212,Paramecium bursaria Chlorella virus CZ-2,AGE52748.1,M1H2Q1,2
4,MGYP003324939103,7SP8,0.030,465,202,37,1,227,44,497,...,0.912,7SP8[A],NaN,2.70,hyaluronan synthase (PBCVCZ2_118R;Cz-2_118R),2.4.1.212,Paramecium bursaria Chlorella virus CZ-2,AGE52748.1,M1H2Q1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,MGYP003341041315,1XV5,0.028,418,114,26,1,146,4,401,...,0.442,1XV5[A],NaN,1.73,DNA α-glucosyltransferase (AGT;α-Gt),2.4.1.26,Tequatrovirus T4,CAA25940.1AAC05390.1AAD42527.1NP_049673.1,D9IE91P04519,72
99,MGYP003388175355,4PHR,0.074,283,183,27,2,244,11,254,...,0.772,4PHR[A],NaN,1.34,"bimodular dual β-glycosyltransferase, N-term G...",2.4.1.-,Streptococcus parasanguinis FW213,ACF35268.1AFJ26875.1,B5A7M0I1ZPA1,101
100,MGYP003624946924,8D3T,0.046,323,194,24,1,278,132,385,...,0.632,"8D3T[A,B,C,D]",D-1-deoxy-GlcpNAc,2.37,"galactan β-1,4-galactosyltransferase (GalS1; G...",2.4.1.-2.4.2.-,Populus trichocarpa,XP_002307008.3,NaN,92
101,MGYP003624946928,5KWK,0.079,392,200,27,1,255,73,440,...,0.912,"5KWK[A,B]",NaN,1.90,"xyloglucan α-1,2-fucosyltransferase 1 (Fut1;Ft...",2.4.1.69,Arabidopsis thaliana,AAD41092.1AAC34480.1AAM98135.1AAO00837.1AAO300...,Q9SWH5W8PV36,37


Our hits contain a lot of fairly poor-quality hits, so let's filter by TM-Score:

In [13]:
best_hits = best_hits[best_hits["TM-Score"] > 0.7]
best_hits.head()

,Query,Target,Identity,Aligned Length,Number of Mismatches,Gap Openings,Query Start,Query End,Target Start,Target End,...,Probability,PDB Code,Ligand,Resolution,Protein Name,EC#,Organism,GenBank,Uniprot,Family
1,MGYP001196446441,6APL,0.161,236,145,13,1,203,78,293,...,0.961,"6APL[A,B,C,D,E,F]",NaN,2.35,"CMP-NeuAc:N-acetylgalactosaminide α-2,6-sialyl...",2.4.3.3,Homo sapiens,AAH40455.1AAA52228.1AAV38775.1CAB61434.1CAS916...,A0A024R8M1Q9UJ37,29
2,MGYP001316708597,7SP8,0.081,370,184,23,1,223,41,401,...,0.981,7SP8[A],NaN,2.70,hyaluronan synthase (PBCVCZ2_118R;Cz-2_118R),2.4.1.212,Paramecium bursaria Chlorella virus CZ-2,AGE52748.1,M1H2Q1,2
3,MGYP001379529187,7SP8,0.074,374,186,24,1,223,41,405,...,0.984,7SP8[A],NaN,2.70,hyaluronan synthase (PBCVCZ2_118R;Cz-2_118R),2.4.1.212,Paramecium bursaria Chlorella virus CZ-2,AGE52748.1,M1H2Q1,2
7,MGYP001324906288,6E4R,0.108,294,191,17,1,239,67,344,...,0.967,"6E4R[A,B]",NaN,2.06,UDP-GalNAc: polypeptide α-N-acetylgalactosamin...,2.4.1.41,Drosophila melanogaster,AAF57964.2AAF57964.3AAF57966.2AAM51988.1ABL756...,Q8MRC9,27
8,MGYP003111233659,6E4R,0.075,331,220,30,1,265,63,373,...,0.967,"6E4R[A,B]",NaN,2.06,UDP-GalNAc: polypeptide α-N-acetylgalactosamin...,2.4.1.41,Drosophila melanogaster,AAF57964.2AAF57964.3AAF57966.2AAM51988.1ABL756...,Q8MRC9,27


What distribution of glycosyltransferase families do we have in our data?

In [17]:
import plotly.express as px
colours = px.colors.qualitative.Prism

fig = px.histogram(best_hits, x="Family", color_discrete_sequence=colours)

fig.update_layout(
    width=1200,
    height=600,
    font=dict(size=16),
    template='plotly_white'
)

fig.show()

Looks like Family 4 is the most common one - there's a lot more reading we could do on this but an initial skim of [this review papepr](https://www.annualreviews.org/doi/10.1146/annurev.biochem.76.061005.092322) says that family 4 is mainly found in Archaea and has the GT-B fold. Glycosyltransferases only tend to have one of two folds - GT-A or GT-B.

## Matching glycosyltransferase MGYPs to encapsulin MGYPs

You'll note that even with this filtered DataFrame we still have ≈55 hits, but we only have 29 Saccharide BGCs. Let's add a column denoting which Saccharide BGC-associated Encapsulin MGYP each glycosyltransferase is associated with.

Unfortunately because of an oversight, the `operon_df_filtered.csv` DataFrame only contains MGYPs 10 genes either side of the Encapsulin MGYP, so some of the glycosyltransferases are too far away and won't be able to be matched with their encapsulin this way.

Let's load the DeepBGC DataFrame and see if we can do it another way:

In [32]:
#We can drop a bunch of unnecessary columns from the CSV file
deepbgc_df = pd.read_csv("../metadata/deepbgc_hits.csv").drop(
    ["detector", "detector_version", "detector_label", "bgc_candidate_id", "antibacterial", "cytotoxic",
     "inhibitor", "antifungal","Alkaloid", "NRP", "Other", "Polyketide", "RiPP", "Saccharide", "Terpene", "Hit?", "Contig"],
axis=1).query("product_class == 'Saccharide'")

deepbgc_df.head()

,Cluster Start,Cluster End,Cluster Length,num_proteins,num_domains,num_bio_domains,deepbgc_score,product_activity,product_class,protein_ids,bio_pfam_ids,pfam_ids,MGYP,ERZ,MGYC,Encapsulin Start,Encapsulin End,Strand,MGYA
0,1545,26938,25393,19,9,2,0.75971,antibacterial,Saccharide,ERZ2772546.64-NODE-64-length-43846-cov-5.80870...,PF04055;PF00534,PF01165;PF07068;PF13884;PF13884;PF00534;PF1369...,MGYP003110882604,ERZ2772546,MGYC001052550128,3131,4792,1,MGYA00587038
8,425,21038,20613,31,16,1,0.92256,antibacterial,Saccharide,ERZ3421540.2866-NODE-2866-length-21688-cov-8.9...,PF00534,PF00166;PF10124;PF13640;PF13759;PF13759;PF1364...,MGYP003662477660,ERZ3421540,MGYC001322393915,3020,3922,1,MGYA00592989
11,3313,15247,11934,15,6,1,0.60569,antibacterial,Saccharide,ERZ2944945.141-NODE-141-length-16210-cov-4.472...,PF00534,PF02557;PF03452;PF17236;PF13524;PF13692;PF00534,MGYP003332394819,ERZ2944945,MGYC001149463221,6101,7063,-1,MGYA00589540
30,0,37790,37790,45,37,5,0.93696,antibacterial,Saccharide,ERZ3421807.161-NODE-161-length-37790-cov-6.082...,PF01041;PF01370;PF02719;PF04321;PF00535,PF13585;PF07068;PF10111;PF00535;PF02562;PF1360...,MGYP003662771788,ERZ3421807,MGYC001322140633,2702,4441,1,MGYA00592987
35,11,57424,57413,45,41,10,0.80509,antibacterial,Saccharide,ERZ2772579.11-NODE-11-length-58335-cov-17.9337...,PF02719;PF01041;PF04321;PF00534;PF01370;PF0865...,PF13551;PF13365;PF13638;PF02562;PF13604;PF1324...,MGYP003131556693,ERZ2772579,MGYC001062169542,45288,46466,1,MGYA00587109


Now, we need to add in the Pfam description labels using the same block of code as before:

In [33]:
import gzip
import json

with open("../DBs/label_descriptions.json.gz", 'rb') as f:
    with gzip.GzipFile(fileobj=f, mode='rb') as gzip_file:
      labels_dict = json.load(gzip_file)

labels_dict["PF19821"] = "Phage capsid protein"
labels_dict["PF19307"] = "Phage capsid-like protein"
labels_dict['PF19289'] = "PmbA/TldA metallopeptidase C-terminal domain"
labels_dict['PF19290'] = "PmbA/TldA metallopeptidase central domain"
labels_dict['PF20211'] = "Family of unknown function (DUF6571)"
labels_dict['PF19782'] = "Family of unknown function (DUF6267)"
labels_dict['PF19343'] = "Family of unknown function (DUF5923)"
labels_dict['PF20036'] = "Major capsid protein 13-like"
labels_dict['PF18960'] = "Family of unknown function (DUF5702)"
labels_dict['PF18906'] = "Phage tail tube protein"
labels_dict['PF19753'] = "Family of unknown function (DUF6240)"

def get_label(pfam):
    try:
        return(labels_dict[pfam])
    except KeyError:
        return(None)
    
pattern = r"Glycosyl\s*transferase"

#This function will access the Pfam family lists for each DataFrame entry and convert it to a free text description
def get_pfam_list_labels(pfam_list):
    pfam_list = pfam_list.split(";")

    try:
        return(" | ".join([get_label(pfam) for pfam in pfam_list]))
    except TypeError:
        return("None")

deepbgc_df["bio_pfam_ids"] = deepbgc_df["bio_pfam_ids"].fillna("None")
deepbgc_df["Descriptions"] = deepbgc_df["bio_pfam_ids"].apply(get_pfam_list_labels)

Now, let's filter down our DataFrame to only include the CDS name of each glycosyltransferase from its contig:

In [34]:
import re

#Make a list of all glycosyltransferase proteins
#This list contains the CDS name from the contig, in the format ERZ.contigname_CDSnumber
proteins = []
for row in deepbgc_df.to_dict(orient="records"):
    mgya = row["MGYA"]
    erz = row["ERZ"]
    mgyp  = row["MGYP"]
    protein_ids = row["protein_ids"].split(";")
    pfam_df = pd.read_csv(f"../deepbgc/{mgya}-{erz}/{mgya}-{erz}.pfam.tsv", sep="\t").query("protein_id in @protein_ids").query("deepbgc_score > 0.5")

    for i, protein in enumerate(protein_ids):
        pfams = pfam_df[pfam_df["protein_id"] == protein]["pfam_id"].values
        if len(pfams) > 0:  
            pfam_labels = " | ".join([get_label(pfam) for pfam in pfams])
        
        if re.match(pattern, pfam_labels):
            proteins.append(protein)

Let's make a dictionary mapping glycosyltransferase MGYP to glycosyltransferase CDS name:

In [35]:
from Bio import SeqIO
from glob import glob

#Load the metadata mapping MGYPs to their ERZ, start, end, and strand data
#We'll also filter this DataFrame to make it easier to process
metadata_df = pd.read_csv("../metadata/cargo_seq_metadata_filtered.csv")

erzs = [protein.split(".")[0] for protein in proteins]
metadata_df = metadata_df.query("ERZ in @erzs")

#Now, we need to map each CDS name to its ERZ, start, end, and strand
#The ERZ is in the CDS name itself but the start, end, and strand data needs to be loaded from the CDS FASTA header
#We can load these FASTA headers from the FASTA files containing all CDSes from each contig
protein_names = [f"{protein.split('_')[1]}_{protein.split('_')[2]}" for protein in proteins]
erz_dict = {}

for protein_name in protein_names:
    erz = protein_name.split(".")[0]
    seq_records = SeqIO.parse(glob(f"../contigs/CDS/*-{erz}_CDS.fasta")[0], "fasta")

    fasta_header = [str(record.description) for record in seq_records if str(record.description).split()[0] == protein_name][0]

    start = fasta_header.split(" # ")[1]
    end = fasta_header.split(" # ")[2]
    strand = fasta_header.split(" # ")[3]

    erz_dict["_".join((erz, start, end, strand))] = protein_name


#And now FINALLY we can make a dictionary mapping the CDS names to MGYPs
mgyp_dict = {}

for _, row in metadata_df.iterrows():
    try:
        mgyp_dict[row["MGYP"]] = erz_dict["_".join((row["ERZ"], str(row["Start"]), str(row["End"]), str(row["Strand"])))]
    except KeyError:
        continue

And FINALLY in this very long-winded and convoluted process, let's write a function to map glycosyltransferase MGYP to encapsulin MGYP, via the DeepBGC DataFrame:

In [40]:
def get_enc_mgyp(cargo_mgyp):
    cargo_cds_name = mgyp_dict[cargo_mgyp]
    return(deepbgc_df[deepbgc_df["protein_ids"].str.contains(cargo_cds_name)]["MGYP"].unique()[0])

And let's add this column to our DataFrame!

In [42]:
best_hits["Encapsulin MGYP"] = best_hits["Query"].apply(get_enc_mgyp)
best_hits.head()

,Query,Target,Identity,Aligned Length,Number of Mismatches,Gap Openings,Query Start,Query End,Target Start,Target End,...,PDB Code,Ligand,Resolution,Protein Name,EC#,Organism,GenBank,Uniprot,Family,Encapsulin MGYP
1,MGYP001196446441,6APL,0.161,236,145,13,1,203,78,293,...,"6APL[A,B,C,D,E,F]",NaN,2.35,"CMP-NeuAc:N-acetylgalactosaminide α-2,6-sialyl...",2.4.3.3,Homo sapiens,AAH40455.1AAA52228.1AAV38775.1CAB61434.1CAS916...,A0A024R8M1Q9UJ37,29,MGYP001178754852
2,MGYP001316708597,7SP8,0.081,370,184,23,1,223,41,401,...,7SP8[A],NaN,2.70,hyaluronan synthase (PBCVCZ2_118R;Cz-2_118R),2.4.1.212,Paramecium bursaria Chlorella virus CZ-2,AGE52748.1,M1H2Q1,2,MGYP001178754852
3,MGYP001379529187,7SP8,0.074,374,186,24,1,223,41,405,...,7SP8[A],NaN,2.70,hyaluronan synthase (PBCVCZ2_118R;Cz-2_118R),2.4.1.212,Paramecium bursaria Chlorella virus CZ-2,AGE52748.1,M1H2Q1,2,MGYP003111298200
7,MGYP001324906288,6E4R,0.108,294,191,17,1,239,67,344,...,"6E4R[A,B]",NaN,2.06,UDP-GalNAc: polypeptide α-N-acetylgalactosamin...,2.4.1.41,Drosophila melanogaster,AAF57964.2AAF57964.3AAF57966.2AAM51988.1ABL756...,Q8MRC9,27,MGYP001412933479
8,MGYP003111233659,6E4R,0.075,331,220,30,1,265,63,373,...,"6E4R[A,B]",NaN,2.06,UDP-GalNAc: polypeptide α-N-acetylgalactosamin...,2.4.1.41,Drosophila melanogaster,AAF57964.2AAF57964.3AAF57966.2AAM51988.1ABL756...,Q8MRC9,27,MGYP003111233400


In [55]:
for name, group in best_hits.groupby("Encapsulin MGYP"):
    print(name)

    for record in group.to_dict(orient="records"):
        print(f"{record['Query']} - {record['Protein Name']} Family {record['Family']} TM-Score {record['TM-Score']} ({record['Organism']}) {record['Uniprot']}")

    print()

MGYP001178754852
MGYP001196446441 - CMP-NeuAc:N-acetylgalactosaminide α-2,6-sialyltransferase II (ST6GalNAc II;Sia7b;SThM) Family 29 TM-Score 0.74 (Homo sapiens) A0A024R8M1Q9UJ37
MGYP001316708597 - hyaluronan synthase (PBCVCZ2_118R;Cz-2_118R) Family 2 TM-Score 0.79 (Paramecium bursaria Chlorella virus CZ-2) M1H2Q1
MGYP001422618815 - sucrose synthase 1 (Sus1;AtSus1;At5g20830) Family 4 TM-Score 0.74 (Arabidopsis thaliana) P49040

MGYP001238560740
MGYP001418685844 - sucrose synthase 1 (Sus1;AtSus1;At5g20830) Family 4 TM-Score 0.78 (Arabidopsis thaliana) P49040
MGYP003123774632 - NY2A_B736L Family 4 TM-Score 0.74 (Paramecium bursaria Chlorella virus NY2A) nan

MGYP001412933479
MGYP001324906288 - UDP-GalNAc: polypeptide α-N-acetylgalactosaminyltransferase (PGANT9A; PGANT9B; CG30463) Family 27 TM-Score 0.75 (Drosophila melanogaster) Q8MRC9
MGYP001383139300 - UDP-GalNAc: polypeptide N-acetylgalactosaminyltransferase 10 (GalNT10;ppGalNAcT10;pp-GalNAc-T10) Family 27 TM-Score 0.73 (Homo sapiens)